# Asignación 3

1 .Descarga el conjunto de datos de educación superior de los Estados Unidos de [www.data.gov/education](https://www.data.gov/education/ ) como un archivo `CSV`. Escribe un programa que reporte a  diez instituciones de educación superior (`HEIs`) que son las más cercanas geográficamente a un punto definido por la latitud media y la longitud media de todas las `HEIs` en el conjunto de datos. Calcular las distancias en grados. Usa `numpy` para el almacenamiento y procesamiento de datos tanto como sea posible. Recuerda que la primera fila del archivo CSV contiene columnas de encabezados y que algunos campos o registros enteros del archivo pueden ser inválidos.

In [ ]:
# Tu codigo
import pandas as pd
import numpy as np
from math import sin, cos, sqrt, asin, pi

R = 6371.0

def distance(pos):
    diff_lat = pos[1] - mean_lat
    diff_long = pos[0] - mean_long
    c = pi/180.0
    return 2.0 * R * asin(sqrt(sin(c * diff_lat / 2) ** 2) + cos(c * mean_lat) * cos(c * pos[1]) * sin(c * (diff_long) / 2) ** 2)

data = pd.read_csv('Dataset_1/data.csv', encoding = "ISO-8859-1")
coord = np.array(data[['LONGITUD', 'LATITUDE']])

# Se calcula la latitud y longitud media de la data
mean_long = np.mean(coord[:, 0])
mean_lat = np.mean(coord[:, 1])

# Aplicar la funcion distance sobre el np.array coord
dis = np.apply_along_axis(distance, 1, coord)

# Se calcula la posicion de los 10 primera valores ordenados
indices = np.argpartition(dis, 10, axis=0)[:10]

# Las 10 instituciones mas cercanas a la media

print("Los 10 institutos de educ sup mas cercanos a la institucion media : ")
print("-------------------------------------------------------")
for i in indices:
    print(data['INSTNM'][i])

2 .Wikipedia tiene muchos datos sobre diversos aspectos demográficos, incluyendo la 
[Lista de paises por consumo de alcohol per capita](https://en.wikipedia.org/wiki/List_of_countries_by_alcohol_consumption_per_capita) y la [Lista de paises por  GDP (PPP) per capita](https://en.wikipedia.org/wiki/List_of_countries_by_GDP_per_capita). 

Escribe un programa que utilice estos datos y realize tabulación cruzada entre el nivel del GDP (por  encima del promedio vs. por debajo del promedio) vs. nivel de consumo de alcohol (por encima del promedio vs. por debajo del promedio). En base a la tabla realizada, ¿las dos medidas están correlacionadas?.

In [ ]:
# Tu codigo
from bs4 import BeautifulSoup
import urllib
import urllib.request

wiki = 'https://en.wikipedia.org/wiki/List_of_countries_by_alcohol_consumption_per_capita'

with urllib.request.urlopen(wiki) as url:
    s = url.read()
    
soup = BeautifulSoup(s)

rank = ''
country = ''
total = ''

table = soup.find('table', {'class': 'wikitable sortable'})

f = open('Dataset_2/alcoholpercapita.csv', 'w')
flag = False

for row in table.findAll('tr'):
    cells = row.findAll('td')
    if len(cells) == 10:
        rank = cells[0].find(text=True)
        country = cells[1].findAll(text=True)
        total = cells[2].find(text=True)
        
    if (not flag):
        write_to_file = 'RANK-OH,PAIS,TOTAL\n'
    else:
        write_to_file = str(rank) + ',' + str(country[1:]).strip('[]') + ',' + str(total) + '\n'
    flag = True
    f.write(write_to_file)
f.close()

In [ ]:
# Scrappeando List_of_countries_by_GDP_(PPP)_per_capita
from bs4 import BeautifulSoup
import urllib
import urllib.request
 
wiki = 'https://en.wikipedia.org/wiki/List_of_countries_by_GDP_(PPP)_per_capita'

with urllib.request.urlopen(wiki) as url:  
    s = url.read()

soup = BeautifulSoup(s)

rank = ''
country = ''
total = ''

table = soup.find('table', { 'class': 'wikitable sortable'})

f = open('Dataset_2/gdppercapita.csv', 'w')
flag = False

for row in table.findAll('tr'):
    cells = row.findAll('td')
    if len(cells) == 3:
        rank = cells[0].find(text=True)
        country = str(cells[1].findAll(text=True)[1:]).replace(',', ' ')
        total = cells[2].find(text=True).replace(',', '.')
    if (not flag):
        write_to_file = 'RANK-GDP,PAIS,INT\n'
    else:
        write_to_file = str(rank) + ',' + str(country[1:]).strip('[]') + ',' + str(total) + '\n'
    flag = True
    f.write(write_to_file)
f.close()


In [ ]:
import pandas as pd

data = pd.read_csv('Dataset_2/alcoholpercapita.csv')
data

In [ ]:
data2 = pd.read_csv('Dataset_2/gdppercapita.csv')
data2

In [ ]:
result = pd.merge(data, data2, on='PAIS')
pd.crosstab(result.INT, result.TOTAL, margins=True)

3 .Combina los datos históricos del consumo de alcohol, con los datos meteorológicos históricos por estado. Utiliza la tabulación cruzada para estimar si los hábitos de consumo de alcohol posiblemente están correlacionados con las temperaturas locales promedio y las precipitaciones totales. En otras palabras, ¿la gente bebe más cuando llueve ?. El sitio web del [Centro Nacional de Datos Climáticos](https://www.ncdc.noaa.gov/cdo-web/)  es un buen punto de partida, para lss búsqueda de datos meteorológicos históricos.

In [ ]:
# Se escogera la cantidad de precipitacion en cada estado en el anho 2016
# Todo la data se consiguio del link propuesto.
# Tu codigo
import pandas as pd
drunk_data = pd.read_csv('drunk_data.csv',error_bad_lines=False, header=None, names=['Country', 'Drunk Driving Deaths' ])

Data={}
a = ''

# "Data" es el diccionario de datos iniciales.
for s in drunk_data['Country']:
    path = 'Datasets/' + s + '.csv'
    Data[s] = pd.read_csv(path, header=None, names=['DAY','CENTERLAT','CENTERLON','data','TEMP'])

In [ ]:
# "Data_mod" ha sido modificada
# Cambiaremos cada fecha por su estado correspondiente para usar
# tabulacion cruzada.
import numpy as np
import math

Data_mod_1 = {}
Data_mod_2 = {}
for s in drunk_data['Country']:
    Data_mod_1[s] = Data[s][:]['DAY'].copy()
    Data_mod_1[s][:] = s # Cambiar fecha por estados.

for s in drunk_data['Country']:
    Data_mod_2[s] = Data[s][:]['TEMP'].copy()    

countries = []
temperatures = []

for s in Data_mod_1:
    for x in Data_mod_1[s] :
        countries.append(x)
    for x in Data_mod_2[s]:
        temperatures.append(x)

a = np.array(countries)
b = np.array(a)
m = crosstab(a, b, rownames=['a'], colnames=['b'])

# FRECUENCIA DE PRECIPITACIONES Y TEMPERATURA PROMEDIO
frecuencias_prec = []
temperatura_prom = []
for s in drunk_data['Country']:
    if s == 'Nevada' or s == 'Oregon':
        frecuencias_prec.append([s, 0]) # No hay precipitacion registradas en esos estados
        temperatura_prom.append([Data_mod_2[s].mean()])
    else:
        frecuencias_prec.append([s, m[s][s]])
        temperatura_prom.append([Data_mod_2[s].mean()])

        temperatura_prom = [[.0] if math.isnan(x[0]) else x for x in temperatura_prom] # Eliminar valores nan

# FRECUENCIA DE MUERTE DE CONDUCTORES EN ESTADO DE EBRIEDAD
drunk_state = []
for s in drunk_data['Drunk Driving Deaths']:
    drunk_state.append([s])
drunk_state

tabla1 = pd.DataFrame(frecuencias_prec)
tabla2 = pd.DataFrame(temperatura_prom)
tabla3 = pd.DataFrame(drunk_state)

tabla = pd.DataFrame(pd.concat([tabla1, tabla2, tabla3], ignore_index=True, axis=1))
tabla 

result = tabla.sort_values([1], ascending=[1])
result

# PODEMOS VER UN CUADRO DONDE 
# COLUMNA 0 : ESTADOS
# COLUMNA 1 : CANTIDAD DE PRECIPITACIONES REGISTRADAS EN UN AÑO
# COLUMNA 2 : TEMPERATURA PROMEDIO EN UN AÑO
# COLUMNA 3 : CANTIDAD DE MUERTES POR CONDUCTORES EN ESTADO DE EBRIEDAD

# SE ORDENO EL CUADRO SEGUN LA CANTIDAD DE PRECIPITACIONES.
# VEMOS QUE LA CANTIDAD DE MUERTES POR ACCIDENTES DE TRANSITO AUMENTA DE 3 CIFRAS A 4 CIFRAS.

# NO PODEMOS DESCARTAR QUE "A MAYOR CANTIDAD DE LLUVIAS, LA GENTE BEBE MAS", TRATANDO DE MODIFICAR
# LA PREGUNTA INICIAL.

# POR LO TANTO, PODRIA EXISTIR UN CORRELACION POSITIVA ENTRE LLUVIAS 
# Y PERSONAS QUE BEBEN.

4 .Escriba un programa que reporte algunas medidas estadísticas básicas de los valores de cierre del índice `S&P500`: la media, la desviación estándar, la asimetría y la correlación entre los valores de cierre y el volumen de comercio en el siglo XXI. ¿Es fiable la correlación?. Puedes descargar los precios históricos desde  [Yahoo! Finanzas](https://finance.yahoo.com/quote/%5EGSPC/history?ltr=1). Recuerda que el siglo XXI comenzó el 1 de enero de 2001.

In [ ]:
# Tu codigo
import numpy as np
import pandas as pd

data = pd.read_csv('Dataset_4/^GSPC.csv', encoding = "ISO-8859-1")
data

In [ ]:
# CALCULO DE MEDIA Y LA DESVIACION ESTANDAR

print("LA MEDIA ES: " + str(data['Close'].mean()))
print("LA DESVIACION ESTANDAR ES: " + str(data['Close'].std()))

In [ ]:
# GRAFICA DE LA 

import matplotlib.pyplot as plt
x = np.array(range(1,4141))
y = np.array(data['Close'])
plt.plot(x, y)
plt.show()


In [ ]:
from scipy import stats

# COEFICIENTE DE ASIMETRIA DE FISHER
a3 = np.sum(y**3) / y.size
a1 = np.sum(y) / y.size
desv = data['Close'].std()

u3 = a3 - 3*a1*(desv**2) + 2*(a1**3)

asi_fisher = u3 / (desv**3)
print(str(asi_fisher) + " es mayor que 0")

# CONCLUIMOS QUE LA ASIMETRIA ES POSITIVA

In [ ]:
# CALCULO DE CORRELACION
vector1 = data['Close']
vector2 = data['Volume']
print(stats.pearsonr(vector1, vector2)[0])

# PRESENTA UNA CORRELACION POSITIVA PERO NO ES FIABLE

5 . Desarrolla  un programa que predice si una ciudad tiene un sistema de metro. Su programa puede necesitar considerar la población, la densidad de población, el tamaño del presupuesto, las condiciones climáticas, los niveles del impuesto sobre la renta y otras variables. Algunos de ellos pueden estar fácilmente disponibles en línea, otros no. Utiliza la regresión logística y un  bosque de decisión aleatorio y escoge  el método que mejor se acomode.

In [ ]:
# Tu codigo

6 .Escribe un programa que agrupa enormes sitios de redes sociales en línea por el número de usuarios registrados y la [clasificación de página global de Alexa](https://en.wikipedia.org/wiki/List_of_social_networking_websites). Debido a que los rangos y los tamaños del sitio varían en un amplio rango, utiliza la escala logarítmica tanto para la agrupación como para la presentación.

In [ ]:
# Tu codigo
# Scrappeando List_of_countries_by_GDP_(PPP)_per_capita
from bs4 import BeautifulSoup
import urllib
import urllib.request
 
wiki = 'https://en.wikipedia.org/wiki/List_of_social_networking_websites'

with urllib.request.urlopen(wiki) as url:  
    s = url.read()

In [ ]:


soup = BeautifulSoup(s)

table = soup.find('table', { 'class': 'wikitable sortable'})
f = open('Dataset_6/socialnetworkingwebsites.csv', 'w')
write_to_file = 'Name,Registered users,Alexa Page Ranking\n'
f.write(write_to_file)
for row in table.findAll('tr'):
    cells = row.findAll('td')

    if len(cells) == 5:
        
        aux1 = cells[2].findAll(text=True) 
        aux2 = cells[4].findAll(text=True)
        
        if len(aux1) > 0 and len(aux2) > 0 and aux2[0] != 'NA':
            
            aux = row.findAll('th')[0].findAll(text=True)
            aux = aux[0].replace(',',' ')
            
            users = aux1[1].split(' ')[0].replace(',','')
            
            rank = aux2[1].replace(',','')
            
            write_to_file = str(aux) + ',' + str(users) + ',' + str(rank) + '\n'
            f.write(write_to_file)

f.close()

In [ ]:
import pandas as pd

data1 = pd.read_csv('Dataset_6/socialnetworkingwebsites.csv')
data1

In [ ]:
import numpy as np
import pylab as pl
import math
from sklearn import neighbors

data2 = np.array(data1).copy()

X = np.array(data2[:, 1], dtype='double')
Y = np.array(data2[:, 2], dtype='double')

X = np.log(X)
Y = np.log(Y)

pl.scatter(X, Y, c='b')
pl.xlabel('Registered users')
pl.ylabel('Alexa Page Ranking')

pl.xlim(X.min(), X.max())
pl.ylim(Y.min(), Y.max())


pl.show()

In [ ]:
import numpy as np
import pylab as pl
import math
from sklearn.cluster import KMeans

data2 = np.array(data1).copy()

X = np.array(data2[:, 1], dtype='double')
Y = np.array(data2[:, 2], dtype='double')

X = np.log(X)
Y = np.log(Y)

# ----------
# K - Means


datos = np.concatenate((X.reshape((X.shape[0], 1)).T, Y.reshape((Y.shape[0], 1)).T), axis=0).T

model = KMeans(n_clusters=2)
model.fit(datos)
model.labels_

# Colorear cada cluster
colormap = np.array(['red', 'black'])

# ----------

pl.scatter(X, Y, c=colormap[model.labels_])
pl.xlabel('Registered users')
pl.ylabel('Alexa Page Ranking')

pl.xlim(X.min(), X.max())
pl.ylim(Y.min(), Y.max())

pl.show()

Una base de datos NoSQL es una colección no volátil de objetos, a menudo conocidos como documentos, con atributos. Se han desarrollado muchas implementaciones diferentes de como almacenar  documentos.  Uno de los más importantes es [MongoDB](https://www.mongodb.com/) que es una base de datos no relacional.

Un servidor MongoDB puede soportar varias bases de datos no relacionadas. Una base de datos consiste en una o más colecciones de documentos. Todos los documentos de una colección tienen identificadores únicos. 
 
Un cliente de Python para  MongoDB se implementa en el módulo `pymongo` como una instancia de la clase `MongoClient`. Se puede crear un cliente sin parámetros (funciona para una instalación de servidor local típica), con el nombre del host y el número de puerto del servidor como parámetros o con el URI (Uniform Resource Identifier) del servidor como parámetro:

7 .Ejecuta las siguientes lineas:

```
import pymongo as mongo
cliente1 = mongo.MongoClient()
cliente2 = mongo.MongoClient("localhost", 27017)
cliente3 = mongo.MongoClient("mongodb://localhost:27017/")
```

In [ ]:
import pymongo as mongo
# MongoClient create una conexion. Si no se especifica argumentos,
# se especifica como "localhost", 27017.
cliente1 = mongo.MongoClient() 

# Por argumentos se especifica la interfaz donde se va a correr,
# ademas del puerto.
cliente2 = mongo.MongoClient("localhost", 27017)

# Se especifica el MongoDB URI completo para definir la conexion.
cliente3 = mongo.MongoClient("mongodb://localhost:27017/")

8 . Explica las siguientes lineas

```
# Se asigna la base de datos llamada dsdb a la variable local db.
db = cliente1.dsdb
# Tambien se puede acceder como si fuera un diccionario.
db = cliente1["dsdb"]
# Se puede asignar a la variable persona la coleccion db.persona de dos # formas
persona = db.persona
persona = db["persona"]

# Creamos 2 diccionarios llamados persona1 y persona2.
persona1 = {"empname" : "Scikit Learn", "dob" : "1957-12-24"}
persona2 = {"_id" : "XVT162", "empname" : "Numpy", "dob" : "1964-05-16"}
# Inserta el diccionario  persona1 en la coleccion persona. Si no 
# existiera la crearia.
# Con la operacion inserted_id retorna el id del diccionario ingresado.
persona_id1 = persona.insert_one(persona1).inserted_id
persona1

# Se obtiene el id del documento persona2 ingresado.
persona_id2 = persona.insert_one(persona2).inserted_id

# Crea un iterable con 2 documentos.
persons = [{"empname" : "Scipy", "dob" : "1809-02-12"},
{"empname" : "MongoDb"}]
# Inserta un iterable de documentos declarado como persons.
resultados = persona.insert_many(persons)
# Devuelve el id de los documentos insertados
resultados.inserted_ids

# Se asigna a la variable todos a todos los documentos incluidos en la 
# coleccion persona.
todos = persona.find()
# Lista los documentos.
list(todos)

# Nos retorna una lista con todas las coincidencias.
list(persona.find({"dob" : "1957-12-24"}))
# Encuentro el primer elemento ingresado, ya que no se le asigno ningun # parametro.
persona.find_one()

# Devuelve la primera coincidencia al argumento pasado.
persona.find_one({"empname" : "Scipy"})
persona.find_one({"_id" : "XVT162"})

# Muestra la cantidad de documentos contenidos en la coleccion.
persona.count()

# Muestra la cantidad de coincidencias con el argumento pasado.
persona.find({"dob": "1957-12-24"}).count()
# De la lista de documentos en la coleccion persona se ordena de acuerdo
# al string "dob"
persona.find().sort("dob")

# Remueve todos los documentos que coincidan con {"dob" : "1957-12-24"}.
resultados = persona.delete_many({"dob" : "1957-12-24"})
# Muestra la cantidad de documentos removidos.
resultados.deleted_count
```